In [1]:
from __future__ import print_function
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import json
from pprint import pprint
import csv

In [2]:
SECRETS_FILE = "authentication.json"
SPREADSHEET = "Candidate Evaluation--Officer (Responses)"

In [3]:
# use creds to create a client to interact with the Google Drive API
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name(SECRETS_FILE, scope)

In [4]:
client = gspread.authorize(creds)

In [5]:
evaluation_officer = client.open(SPREADSHEET)

In [6]:
officer_sheet = evaluation_officer.sheet1

In [7]:
officer_data = officer_sheet.get_all_records()

In [8]:
CANDIDATES = ["Xuezheng Wang", "Chen Ling", "Shuyang Miao", "Shantao Yi", 
              "Cheng Gong", "Wanju Gu", "Zhixiong Li", "Zixin Zeng", 
              "Weiwen Hong", "Ziyan Cui", "Weihao Zeng"]

In [9]:
DEPT = ["Strategy", "Technology", "Internal Affairs", "Career Development", "Inspiration", 
        "Business Development", "Meida", "New Dept./Committee"]

In [10]:
LB_MEMBERS = ["Zhiyuan Chen", "Shuyang Liu", "Ruolei Dong", "Xinyu Liu", 
              "Chen Ling", "Zhixiong Li", "Zhizhen Qin", "Shantao Yi"]

In [11]:
LB_TO_DEPTS = {
    "Zhixiong Li": ["Internal Affairs", "Inspiration"],
    "Shantao Yi": ["Career Development"],
    "Chen Ling": ["Inspiration"],
    "Zhiyuan Chen": ["Business Development"],
}

In [12]:
candidate_comments = []
for d in officer_data:
    candidate_comments.append({
        "candidate": d['Candidate\'s Name'],
        "department": d['Election Department'],
        "recommend": d['Do you recommend this person overall?'],
        "comment": d['Please write some comments on the candidate you are reviewing (200 characters minimum). ']
    })

In [13]:
candidate_comments = sorted(candidate_comments, key=lambda elem: "%s %s" % (elem['department'], elem['candidate'])) 

In [14]:
for l in LB_MEMBERS:
    if l in LB_TO_DEPTS:
        comment_for_l = []
        for comment in candidate_comments:
            if comment['candidate'] != l and comment['department'] not in LB_TO_DEPTS[l]:
                comment_for_l.append(comment)
    else:
        comment_for_l = candidate_comments
    
        
    with open("{}.csv".format(l), 'wb') as f:
        keys = ['candidate', 'department', 'comment', 'recommend']
        dict_writer = csv.DictWriter(f, keys)
        dict_writer.writeheader()
        dict_writer.writerows(comment_for_l)